In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import yfinance as yf

pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

In [ ]:
def plot(df, ticker, x_label, y_label):
    plt.figure(figsize=(10,5))
    plt.plot(df.index, df['Close_Price'], linestyle="-", color='orange')
    plt.title(f'{ticker} over time')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.show()

In [ ]:
# Import the senticrypt data 

SENTICRYPT_ENDPOINT = "https://api.senticrypt.com/v2/all.json"

r = requests.get(SENTICRYPT_ENDPOINT)

print(f"Issue with requests: {r.raise_for_status()}\nStatus code: {r.status_code}")

senticrypt_json = r.json()

df_raw_sc = pd.DataFrame(senticrypt_json)

Issue with requests: None
Status code: 200


In [ ]:
df_raw_sc['date'] = pd.to_datetime(df_raw_sc['date'])
df_raw_sc = df_raw_sc.drop('price', axis=1) 
df_raw_sc.set_index('date', inplace=True)
df_raw_sc.rename(columns={ "volume": "Senticrypt_Volume", "score1": "Senticrypt_Score_1", "score2": "Senticrypt_Score_2", "score3": "Senticrypt_Score_3", "count": "Senticrypt_Count", "mean": "Senticrypt_Mean", "sum": "Senticrypt_Sum",
                         }, inplace=True)
df_raw_sc.head()

,Senticrypt_Volume,Senticrypt_Score_1,Senticrypt_Score_2,Senticrypt_Score_3,Senticrypt_Count,Senticrypt_Mean,Senticrypt_Sum
date,,,,,,,
2024-02-06,1950.0,0.9136,-0.1280,-0.3125,3,0.1577,0.4731
2024-02-05,1608.0,-0.1428,0.2364,0.2647,210,0.1194,25.0841
2024-02-04,1108.0,-0.2704,0.0852,0.1755,123,-0.0032,-0.3944
2024-02-03,1608.0,-0.2593,0.2605,0.2420,123,0.0811,9.9717
2024-02-02,8303.0,-0.0255,0.2249,0.2148,123,0.1381,16.9832


In [ ]:
tickers = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'ADA-USD', 'MATIC-USD', 'USDT-USD', 'XRP-USD', 'USDC-USD', 'DOGE-USD', 'SOL-USD', 'GC=F', 'BZ=F','WTI', 'NG=F', 'MSTR', 'GLXY.TO', 'TSLA', 'MARA','SQ','VYGVQ', '^FTSE','^DJI','^IXIC', '^BSESN','^HSI','^N225','^FTMC', 'DX-Y.NYB', 'EXY', 'USDCNY=X','USDJPY=X','GBPUSD=X', '^TNX']
names = ['BTC', 'ETH', 'BNB', 'ADA', 'POL', 'USDT', 'XRP', 'USDC', 'DGC', 'SOL', 'Gold', 'Crude_Oil_Brent','Crude_Oil_Wti', 'Natural_Gas', 'MSTR', 'GLXY', 'TSLA', 'MARA','SQ','VYGVQ', 'FTSE100','DJ','Nasdaq','Sensex', 'Hangseng', 'Nikkei', 'FTSE250', 'USD_Index', 'EURO_Index','USD_CNY','USD_JPY', 'GBP_USD','US_Treasury_Interest_Rate_%']
# bitcoin = yf.download('BTC-USD', start='2014-01-01')
# euthereum = yf.download('ETH-USD', start='')

crypto = {}
for i,j in zip(tickers, names):
    df = yf.download(i)
    df = df.rename(columns={'Close': 'Close_Price', 'Volume':'Vol.'})
    df = df.drop('Adj Close', axis=1)
    crypto[j] = df 

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
crypto.keys()

dict_keys(['BTC', 'ETH', 'BNB', 'ADA', 'POL', 'USDT', 'XRP', 'USDC', 'DGC', 'SOL', 'Gold', 'Crude_Oil_Brent', 'Crude_Oil_Wti', 'Natural_Gas', 'MSTR', 'GLXY', 'TSLA', 'MARA', 'SQ', 'VYGVQ', 'FTSE100', 'DJ', 'Nasdaq', 'Sensex', 'Hangseng', 'Nikkei', 'FTSE250', 'USD_Index', 'EURO_Index', 'USD_CNY', 'USD_JPY', 'GBP_USD', 'US_Treasury_Interest_Rate_%'])

In [ ]:
# merge data 

# add the ticker to the column name 

for i in crypto.keys():
    column_list = []
    for j in crypto[i].columns:
        column_list.append(f'{i}_{j}')
    crypto[i].columns = column_list

In [ ]:
# read in API's
# create endpoint of where bot would stop for number of daily transactions
N_TRANSACTION_ENDPOINT = "https://api.blockchain.info/charts/n-transactions"

# create dictionary of what data to pull - the API info will give you hint on what to create as dictionary
payload = {"timespan": "12years",
            "start": "2014-01-01",
            "period":"day", 
            'sampled': 'false'
            }

# perform request
r = requests.get(N_TRANSACTION_ENDPOINT, params=payload)

# print if any status issues
print(r.raise_for_status())

# print status code (200 means ok)
print(r.status_code)

# print the web address it went to to get the data
print(r.url)

#assign the json file to variable data
data = r.json()


df_num_trans = pd.DataFrame(data["values"])
df_num_trans['x'] = pd.to_datetime(df_num_trans['x'], unit='s')
df_num_trans.set_index('x', inplace=True)
df_num_trans = df_num_trans.rename(columns={'y':'Total_Num_Transactions_Per_Day'})
df_num_trans

None
200
https://api.blockchain.info/charts/n-transactions?timespan=12years&start=2014-01-01&period=day&sampled=false


,Total_Num_Transactions_Per_Day
x,
2014-01-01,43070.0
2014-01-02,54770.0
2014-01-03,60980.0
2014-01-04,52052.0
2014-01-05,58662.0
...,...
2024-02-01,413272.0
2024-02-02,429732.0
2024-02-03,515539.0


In [ ]:
N_TRANSACTION_ENDPOINT = "https://api.blockchain.info/charts/n-unique-addresses"

# create dictionary of what data to pull - the API info will give you hint on what to create as dictionary
payload = {"timespan": "12years",
            "start": "2014-01-01",
            "period":"day", 
            'sampled': 'false'
            }


# perform request
r = requests.get(N_TRANSACTION_ENDPOINT, params=payload)

# print if any status issues
print(r.raise_for_status())

# print status code (200 means ok)
print(r.status_code)

# print the web address it went to to get the data
print(r.url)

#assign the json file to variable data
data = r.json()


df_addresses_unique = pd.DataFrame(data["values"])
df_addresses_unique['x'] = pd.to_datetime(df_addresses_unique['x'], unit='s')
df_addresses_unique.set_index('x', inplace=True)
df_addresses_unique = df_addresses_unique.rename(columns={'y':'Tot_Num_Unique_Address_BTC_Per_Day'})

None
200
https://api.blockchain.info/charts/n-unique-addresses?timespan=12years&start=2014-01-01&period=day&sampled=false


In [ ]:
# merge df_num_trans and number of transactions 


In [ ]:
df = pd.DataFrame(crypto['BTC'])

for i in crypto.keys():
    if i == 'BTC':
        pass
    else:
        df = pd.merge(df, crypto[i], how='left', left_index=True, right_index=True)

df = pd.merge(df, df_raw_sc, how='left', left_index=True, right_index=True)

df = pd.merge(df,df_addresses_unique, how='left', left_index=True, right_index=True)

df = pd.merge(df,df_num_trans, how='left', left_index=True, right_index=True) 

In [ ]:
df.to_csv('../output/base.csv')

In [ ]:
df.tail(50)

,BTC_Open,BTC_High,BTC_Low,BTC_Close_Price,BTC_Vol.,ETH_Open,ETH_High,ETH_Low,ETH_Close_Price,ETH_Vol.,BNB_Open,BNB_High,BNB_Low,BNB_Close_Price,BNB_Vol.,ADA_Open,ADA_High,ADA_Low,ADA_Close_Price,ADA_Vol.,POL_Open,POL_High,POL_Low,POL_Close_Price,POL_Vol.,USDT_Open,USDT_High,USDT_Low,USDT_Close_Price,USDT_Vol.,XRP_Open,XRP_High,XRP_Low,XRP_Close_Price,XRP_Vol.,USDC_Open,USDC_High,USDC_Low,USDC_Close_Price,USDC_Vol.,DGC_Open,DGC_High,DGC_Low,DGC_Close_Price,DGC_Vol.,SOL_Open,SOL_High,SOL_Low,SOL_Close_Price,SOL_Vol.,Gold_Open,Gold_High,Gold_Low,Gold_Close_Price,Gold_Vol.,Crude_Oil_Brent_Open,Crude_Oil_Brent_High,Crude_Oil_Brent_Low,Crude_Oil_Brent_Close_Price,Crude_Oil_Brent_Vol.,Crude_Oil_Wti_Open,Crude_Oil_Wti_High,Crude_Oil_Wti_Low,Crude_Oil_Wti_Close_Price,Crude_Oil_Wti_Vol.,Natural_Gas_Open,Natural_Gas_High,Natural_Gas_Low,Natural_Gas_Close_Price,Natural_Gas_Vol.,MSTR_Open,MSTR_High,MSTR_Low,MSTR_Close_Price,MSTR_Vol.,...,VYGVQ_Vol.,FTSE100_Open,FTSE100_High,FTSE100_Low,FTSE100_Close_Price,FTSE100_Vol.,DJ_Open,DJ_High,DJ_Low,DJ_Close_Price,DJ_Vol.,Nasdaq_Open,Nasdaq_High,Nasdaq_Low,Nasdaq_Close_Price,Nasdaq_Vol.,Sensex_Open,Sensex_High,Sensex_Low,Sensex_Close_Price,Sensex_Vol.,Hangseng_Open,Hangseng_High,Hangseng_Low,Hangseng_Close_Price,Hangseng_Vol.,Nikkei_Open,Nikkei_High,Nikkei_Low,Nikkei_Close_Price,Nikkei_Vol.,FTSE250_Open,FTSE250_High,FTSE250_Low,FTSE250_Close_Price,FTSE250_Vol.,USD_Index_Open,USD_Index_High,USD_Index_Low,USD_Index_Close_Price,USD_Index_Vol.,EURO_Index_Open,EURO_Index_High,EURO_Index_Low,EURO_Index_Close_Price,EURO_Index_Vol.,USD_CNY_Open,USD_CNY_High,USD_CNY_Low,USD_CNY_Close_Price,USD_CNY_Vol.,USD_JPY_Open,USD_JPY_High,USD_JPY_Low,USD_JPY_Close_Price,USD_JPY_Vol.,GBP_USD_Open,GBP_USD_High,GBP_USD_Low,GBP_USD_Close_Price,GBP_USD_Vol.,US_Treasury_Interest_Rate_%_Open,US_Treasury_Interest_Rate_%_High,US_Treasury_Interest_Rate_%_Low,US_Treasury_Interest_Rate_%_Close_Price,US_Treasury_Interest_Rate_%_Vol.,Senticrypt_Volume,Senticrypt_Score_1,Senticrypt_Score_2,Senticrypt_Score_3,Senticrypt_Count,Senticrypt_Mean,Senticrypt_Sum,Tot_Num_Unique_Address_BTC_Per_Day,Total_Num_Transactions_Per_Day
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-12-19,42641.511719,43354.296875,41826.335938,42270.527344,23171001281,2218.240967,2253.348877,2139.723389,2177.872559,1.061918e+10,241.347687,254.574585,241.347687,252.637192,1.254352e+09,0.601132,0.616213,0.568203,0.576600,6.619067e+08,0.800329,0.821124,0.754894,0.765359,4.936204e+08,0.999771,1.000601,0.999282,0.999890,4.465701e+10,0.612508,0.615832,0.598158,0.605456,1.137148e+09,1.000001,1.000937,0.999696,1.000257,5.304568e+09,0.092038,0.093355,0.089607,0.090297,4.824528e+08,74.349190,76.271408,72.010612,72.989769,1.900875e+09,2025.199951,2042.699951,2025.199951,2038.400024,498.0,78.050003,79.660004,77.430000,79.230003,25735.0,3.25,3.28,3.18,3.27,2321500.0,2.519,2.565,2.385,2.492,170435.0,583.500000,590.409973,564.510010,571.299988,873800.0,...,117216.0,7614.500000,7642.200195,7605.399902,7638.000000,9.618673e+08,37311.820312,37562.828125,37311.820312,37557.921875,272740000.0,14923.019531,15003.799805,14921.599609,15003.219727,5.757180e+09,71479.281250,71623.710938,71071.976562,71437.187500,8600.0,16551.439453,16579.550781,16408.019531,16505.000000,1.521453e+09,32774.210938,33219.390625,32654.429688,33219.390625,106300000.0,19220.599609,19342.800781,19220.199219,19316.000000,366939800.0,102.470001,102.629997,102.070000,102.169998,0.0,NaN,NaN,NaN,NaN,NaN,7.1320,7.1463,7.0585,7.1320,0.0,142.722000,144.934998,142.263000,142.722000,0.0,1.265342,1.276145,1.265022,1.265182,0.0,3.898,3.928,3.896,3.922,0.0,3009.0,-0.3653,0.3051,0.1785,159.0,0.0394,6.2662,684666.0,662880.0
2023-12-20,42261.300781,44275.585938,42223.816406,43652.250000,27868908174,2177.239502,2263.329102,2161.218750,2201.911377,1.265229e+10,252.629761,261.557404,250.765045,260.107361,1.053280e+09,0.